# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

## Notebook 1

First, downloading all the dependencies:

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Install lxml for scraping Wikipedia

In [5]:
pip install lxml

     |████████████████████████████████| 5.5MB 17.9MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


Read in the Wikipedia page on the list of postal codes of Canada. Created a Pandas DataFrame to hold the data. Pro-processing included removing rows with 'Not assigned' Boroughs. 

In [38]:
link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

canada_page = pd.read_html(link)

canada_codes = pd.DataFrame(canada_page[0].iloc[0:])

canada_codes.columns = ['PostalCode','Borough','Neighborhood']

canada_codes = canada_codes[canada_codes.Borough != 'Not assigned']
canada_codes = canada_codes.sort_values(['PostalCode'])
canada_codes = canada_codes.reset_index(drop=True)

canada_codes.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [30]:
canada_codes.shape

(103, 3)

# Notebook 2

Now that we have built a dataframe of the postal codes for each neighborhood along borough and neighborhood name, we will include map coordinates to our dataframe. In order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In [7]:
geospatial_coordiantes = pd.read_csv('Geospatial_Coordinates.csv')

In [50]:
geospatial_coordiantes.columns = ['PostalCode2','Latitude','Longitude']

geospatial_coordiantes = geospatial_coordiantes.sort_values(['PostalCode2'])

combined_canada_data = pd.concat([canada_codes, geospatial_coordiantes], axis=1, sort=False)
combined_canada_data = combined_canada_data.drop(columns='PostalCode2')
combined_canada_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
